# Лабораторная работа № 7

Классификация обзоров фильмов

Выполнил:
    Студент группы БФИ1901
    Чернышов Дмитрий
    
Задачи:

   1. Ознакомиться с рекуррентными нейронными сетями
   2. Изучить способы классификации текста
   3. Ознакомиться с ансамблированием сетей
   4. Построить ансамбль сетей, который позволит получать точность не менее 97%

# Цель работы:
Классификация последовательностей - это проблема прогнозирующего моделирования,
когда у вас есть некоторая последовательность входных данных в пространстве или
времени, и задача состоит в том, чтобы предсказать категорию для последовательности.
Проблема усложняется тем, что последовательности могут различаться по длине,
состоять из очень большого словарного запаса входных символов и могут потребовать от
модели изучения долгосрочного контекста или зависимостей между символами во входной
последовательности.
В данной лабораторной работе также будет использоваться датасет IMDb, однако
обучение будет проводиться с помощью рекуррентной нейронной сети.

In [1]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import numpy as np
from tkinter import *
from tkinter import ttk
import tkinter as tk
from tensorflow.keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [2]:
from keras.datasets import imdb #Загружаем датесет IMDb,
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),
axis=0)

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) #длина отзыва 500 слов
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32 #на вход получает номера слов, а на выходе выдаёт их векторные представления 

model = Sequential()
#векторизуем
model.add(Embedding(top_words, embedding_vecor_length,
input_length=max_review_length))
#LSTM разработаны специально, чтобы избежать проблемы долговременной зависимости. 100 единицами памяти
model.add(LSTM(100)) 
#поскольку это проблема классификации, мы используем плотный
#выходной слой с одним нейроном и сигмоидной функцией активации, чтобы сделать 0 или
#1 прогноз для двух классов (хорошего и плохого) в задаче.
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 280s 712ms/step - loss: 0.4558 - accuracy: 0.7813 - val_loss: 0.6333 - val_accuracy: 0.7905
Epoch 2/3
391/391 [==============================] - 247s 631ms/step - loss: 0.2892 - accuracy: 0.8855 - val_loss: 0.3054 - val_accuracy: 0.8738
Epoch 3/3
391/391 [

In [ ]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)

targets = np.concatenate((training_targets, testing_targets),
axis=0)

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train,
maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,
maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length,
input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) #filters - Количество выходных фильтров в свертке
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test),
epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Архитектура сети со слоями Dropout

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)

targets = np.concatenate((training_targets, testing_targets),
axis=0)

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train,
maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,
maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) #filters - Количество выходных фильтров в свертке
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(MaxPooling1D(pool_size=2))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(LSTM(100))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=32)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from keras.datasets import imdb
(training_data, training_targets), (testing_data,
testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)

targets = np.concatenate((training_targets, testing_targets),
axis=0)

top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train,
maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test,
maxlen=max_review_length)

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) #filters - Количество выходных фильтров в свертке
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(MaxPooling1D(pool_size=2))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(LSTM(100))
model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
from keras.datasets import imdb #Загружаем датесет IMDb,
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

top_words = 5000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

X_test = data[:250]
y_test = targets[:250]
X_train = data[1:]
y_train = targets[1:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32 # длина вектора, в котором будут представляться слова

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length)) # плотный вектор
# input_length - размер входных данных
model.add(LSTM(100)) # рекуррентный слой
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
from keras.datasets import imdb #Загружаем датесет IMDb,
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

top_words = 5000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=top_words)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

X_test = data[:270] 
y_test = targets[:270]
X_train = data[1:]
y_train = targets[1:]

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

embedding_vecor_length = 32 #на вход получает номера слов, а на выходе выдаёт их векторные представления 

model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')) # сверточный слой
model.add(MaxPooling1D(pool_size=2)) # субдискретизирующий слой
model.add(LSTM(100)) # рекуррентный слой
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [12]:
index = imdb.get_word_index()
def predict(txt:str):
    txt = txt.lower()
    txt1 = ""
    for i in txt:
        if('a'<=i<='z' or i==' '):
            txt1+=i
    txt1=txt1.split()
    tokens=np.array([min(index.get(i, 5000),5000)+3 for i in txt1])
    vector = sequence.pad_sequences([tokens], maxlen=max_review_length)
    p=model.predict(vector)[0][0]
    return "POSITIVE "+str(p) if p>0.5 else "NEGATIVE "+str(1-p)

predict(input())

Great movie really liked the actor in the title role, good quality


'POSITIVE 0.89095867'